In [1]:
import pandas as pd
import numpy as np
import scipy
import statistics as st
from statistics import mean
from statistics import median
from statistics import variance
from scipy import stats
from scipy.stats import mstats
import math
import statsmodels.api as sm 
from statsmodels.formula.api import ols



In [2]:
data = pd.read_csv (r"C:\Data\tablets1.txt")
print(data)

   'Lab1'  ''Lab2"'  'Lab3'  'Lab4'  'Lab5'  'Lab6'  'Lab7'
0    4.13      3.86    4.00    3.88    4.02    4.02    4.00
1    4.07      3.85    4.02    3.88    3.95    3.86    4.02
2    4.04      4.08    4.01    3.91    4.02    3.96    4.03
3    4.07      4.11    4.01    3.95    3.89    3.97    4.04
4    4.05      4.08    4.04    3.92    3.91    4.00    4.10
5    4.04      4.01    3.99    3.97    4.01    3.82    3.81
6    4.02      4.02    4.03    3.92    3.89    3.98    3.91
7    4.06      4.04    3.97    3.90    3.89    3.99    3.96
8    4.10      3.97    3.98    3.97    3.99    4.02    4.05
9    4.04      3.95    3.98    3.90    4.00    3.93    4.06


In [3]:
data.columns = ['lab1', 'lab2','lab3','lab4','lab5','lab6','lab7'] # 
print (data)

   lab1  lab2  lab3  lab4  lab5  lab6  lab7
0  4.13  3.86  4.00  3.88  4.02  4.02  4.00
1  4.07  3.85  4.02  3.88  3.95  3.86  4.02
2  4.04  4.08  4.01  3.91  4.02  3.96  4.03
3  4.07  4.11  4.01  3.95  3.89  3.97  4.04
4  4.05  4.08  4.04  3.92  3.91  4.00  4.10
5  4.04  4.01  3.99  3.97  4.01  3.82  3.81
6  4.02  4.02  4.03  3.92  3.89  3.98  3.91
7  4.06  4.04  3.97  3.90  3.89  3.99  3.96
8  4.10  3.97  3.98  3.97  3.99  4.02  4.05
9  4.04  3.95  3.98  3.90  4.00  3.93  4.06


In [4]:
lab1=  data['lab1']
lab2 = data['lab2']
lab3 = data['lab3']
lab4 = data['lab4']
lab5 = data['lab5']
lab6 = data['lab6']
lab7 = data['lab7']

amount = pd.concat([lab1, lab2, lab3, lab4, lab5, lab6, lab7])

x = range(1,8,1)
lab = np.repeat(x, [10, 10,10,10,10,10,10], axis=0)
print(lab)



[1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7]


In [5]:
y = np.column_stack((amount, lab)) 
#print(y)

newdata = pd.DataFrame(y, columns = ['amount', 'lab'])
print(newdata)

newdata['lab'] = newdata['lab'].astype('category') 
# this is to declare with Python that column "lab" in the dataframe "newdata" is a factor

    amount  lab
0     4.13  1.0
1     4.07  1.0
2     4.04  1.0
3     4.07  1.0
4     4.05  1.0
..     ...  ...
65    3.81  7.0
66    3.91  7.0
67    3.96  7.0
68    4.05  7.0
69    4.06  7.0

[70 rows x 2 columns]


In [7]:
###############    ANOVA 
import statsmodels.api as sm 
from statsmodels.formula.api import ols

mod = ols('amount ~ lab', data=newdata).fit()
anova = sm.stats.anova_lm(mod, typ=1) # typ could be 1, 2 or 3. 
print(anova)


            df    sum_sq   mean_sq         F    PR(>F)
lab        6.0  0.124737  0.020790  5.660069  0.000095
Residual  63.0  0.231400  0.003673       NaN       NaN


In [16]:
###############    ANOTHER WAY TO GET ANOVA 
anova2 = scipy.stats.f_oneway(lab1, lab2, lab3, lab4, lab5, lab6, lab7)
print(anova2)
# this is equivalent to the above but less information

F_onewayResult(statistic=5.66006914433881, pvalue=9.452826448501141e-05)


In [17]:
####  KRUSKALL-WALLIS TEST
from scipy.stats import kruskal
kruskal = scipy.stats.kruskal(lab1, lab2, lab3, lab4, lab5, lab6, lab7)

print(kruskal)


KruskalResult(statistic=29.605984376371413, pvalue=4.6701021928163535e-05)


In [20]:
##########  BONFERRONI TEST (each pair is compared by a 2-independent sample t-test)
import statsmodels.stats.multicomp as mc

comparison = mc.MultiComparison(newdata['amount'], newdata['lab'])
result, tbl1, tbl2 = comparison.allpairtest(stats.ttest_ind, method= "bonf", alpha = 0.1) # alpha = 0.05 is the default

print(result)
# Bonf = comparison.allpairtest(stats.ttest_ind, method= "bonf")
# the 2-independent sample t-test for each pair is conducted with equal variances.

Test Multiple Comparison ttest_ind 
FWER=0.10 method=bonf
alphacSidak=0.01, alphacBonf=0.005
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
   1.0    2.0  2.1538 0.0451    0.9462  False
   1.0    3.0  4.6692 0.0002     0.004   True
   1.0    4.0  9.6322    0.0       0.0   True
   1.0    5.0   5.046 0.0001    0.0018   True
   1.0    6.0  4.5392 0.0003    0.0053   True
   1.0    7.0  2.2271 0.0389    0.8177  False
   2.0    3.0 -0.2048   0.84       1.0  False
   2.0    4.0  2.5446 0.0203    0.4269  False
   2.0    5.0  1.1892 0.2498       1.0  False
   2.0    6.0   1.186  0.251       1.0  False
   2.0    7.0 -0.0256 0.9798       1.0  False
   3.0    4.0  6.4703    0.0    0.0001   True
   3.0    5.0  2.3591 0.0298    0.6263  False
   3.0    6.0  2.1405 0.0463    0.9715  False
   3.0    7.0  0.1798 0.8593       1.0  False
   4.0    5.0 -1.7682  0.094       1.0  False
   4.0    6.0 -1.4783 0.1566       1.0  False
   4.0    7.0 -2.7063 0.0145    0

In [28]:
t, p = stats.ttest_ind(lab1,lab2) # by default, equal variance and 2 sided test is used
print('test statitsics is: ',t) 
print('p-value is: ',p)
# print( stats.ttest_ind(lab1,lab2) )

test statitsics is:  2.153808783913392
p-value is:  0.04505639785202304


In [29]:
import statsmodels.api
statsmodels.stats.weightstats.ttest_ind(lab1, lab2, alternative='two-sided', usevar='pooled', value=0)
# usevar = ‘pooled’ or ‘unequal’
# link:  https://www.statsmodels.org/dev/generated/statsmodels.stats.weightstats.ttest_ind.html

(2.153808783913392, 0.04505639785202304, 18.0)

In [14]:
##########  BONFERRONI TEST (each paired is compared by a Mann_Whitney U test)
comparison = mc.MultiComparison(newdata['amount'], newdata['lab'])
result, tbl1, tbl2 = comparison.allpairtest(stats.mannwhitneyu, method= "bonf") # alpha = 0.05 is the default
print(result)

Test Multiple Comparison mannwhitneyu 
FWER=0.05 method=bonf
alphacSidak=0.00, alphacBonf=0.002
group1 group2 stat  pval  pval_corr reject
------------------------------------------
   1.0    2.0 28.0 0.0512       1.0  False
   1.0    3.0  4.0 0.0003    0.0057   True
   1.0    4.0  0.0 0.0001    0.0018   True
   1.0    5.0  1.0 0.0001    0.0024   True
   1.0    6.0  1.0 0.0001    0.0025   True
   1.0    7.0 22.5 0.0201    0.4212  False
   2.0    3.0 46.5   0.41       1.0  False
   2.0    4.0 23.5 0.0243    0.5103  False
   2.0    5.0 34.0 0.1197       1.0  False
   2.0    6.0 33.0 0.1055       1.0  False
   2.0    7.0 50.0 0.4849       1.0  False
   3.0    4.0  1.0 0.0001    0.0025   True
   3.0    5.0 29.0 0.0596       1.0  False
   3.0    6.0 25.5 0.0342     0.718  False
   3.0    7.0 40.0 0.2358       1.0  False
   4.0    5.0 32.0  0.092       1.0  False
   4.0    6.0 26.0 0.0374    0.7852  False
   4.0    7.0 17.5 0.0077    0.1614  False
   5.0    6.0 49.0 0.4848       1.0  False
 

In [12]:
##########  TUKEY  TEST:

import statsmodels.stats.multicomp as mc

comparison = mc.MultiComparison(newdata['amount'], newdata['lab'])
tukey = comparison.tukeyhsd(alpha = 0.1) # alpha = 0.05 is the default
print(tukey.summary() )


Multiple Comparison of Means - Tukey HSD, FWER=0.10 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
   1.0    2.0   -0.065 0.2166 -0.1399  0.0099  False
   1.0    3.0   -0.059 0.3225 -0.1339  0.0159  False
   1.0    4.0   -0.142  0.001 -0.2169 -0.0671   True
   1.0    5.0   -0.105 0.0046 -0.1799 -0.0301   True
   1.0    6.0   -0.107 0.0036 -0.1819 -0.0321   True
   1.0    7.0   -0.064 0.2324 -0.1389  0.0109  False
   2.0    3.0    0.006    0.9 -0.0689  0.0809  False
   2.0    4.0   -0.077 0.0833 -0.1519 -0.0021   True
   2.0    5.0    -0.04 0.7317 -0.1149  0.0349  False
   2.0    6.0   -0.042 0.6894 -0.1169  0.0329  False
   2.0    7.0    0.001    0.9 -0.0739  0.0759  False
   3.0    4.0   -0.083 0.0479 -0.1579 -0.0081   True
   3.0    5.0   -0.046 0.6049 -0.1209  0.0289  False
   3.0    6.0   -0.048 0.5627 -0.1229  0.0269  False
   3.0    7.0   -0.005    0.9 -0.0799  0.0699  False
   4.0    5.0    0.037  0.795 -0.0379  0.1119 

In [16]:
#############   Equal Variances TEST:

#Bartlett test (when samples are assumed normality):
Bar = scipy.stats.bartlett(lab1, lab2, lab3, lab4, lab5, lab6, lab7)
print(Bar)

BartlettResult(statistic=24.369716004698912, pvalue=0.0004464732077665512)


In [17]:
#############   Equal Variances TEST:

#Levene test (when distribution of samples are unknown)
Levene = scipy.stats.levene(lab1, lab2, lab3, lab4, lab5, lab6, lab7)
print(Levene)

LeveneResult(statistic=2.2444617178390893, pvalue=0.050181442540930994)


In [18]:
##### Normality checking :

# Shappiro Wilk Test (used for our course):
from scipy.stats import shapiro
Shapiro1 = scipy.stats.shapiro(amount) # for the variable amount
print(Shapiro1)
Shapiro2 = scipy.stats.shapiro(mod.resid) # for the residuals of Anova
print(Shapiro2)

(0.9744956493377686, 0.16336943209171295)
(0.952541708946228, 0.0098727997392416)


In [19]:
##### normality checking :
# Kolmogorov Smirnov Test for the variable amount
test = np.random.normal(mean(amount), np.std(amount), 70 )
KS = scipy.stats.ks_2samp(amount,test)
print(KS)

Ks_2sampResult(statistic=0.12857142857142856, pvalue=0.5795548581933454)


In [20]:
##### normality checking :
# Kolmogorov Smirnov Test for the residuals
res = np.random.normal(mean(mod.resid), np.std(mod.resid), 70 ) #(***)
KS_res = scipy.stats.ks_2samp(mod.resid,res)
print(KS_res) # p-value can be different due to the randomness in (***)

Ks_2sampResult(statistic=0.08571428571428574, pvalue=0.9504984566743319)
